In [1]:
import sys
sys.path.append('../scripts/')

In [1]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
from altairPlotting import make_sky_map, make_alts_plot, get_data, get_moon_data, get_interactive_elements

import warnings
warnings.simplefilter(action='ignore')

In [3]:
df = pd.read_csv('../data/mjd-59418-sdss-simple-expanded.csv', index_col=0)
data, star_data = get_data(df)
moon_pos = pd.read_csv('../data/moon-positions-mjd-59418.csv')

select_field, select_time, field_scale = get_interactive_elements()

sky = make_sky_map(data, star_data, moon_pos, select_field, select_time, field_scale)
altitudes = make_alts_plot(data, select_field, select_time, field_scale)

chart = (sky & altitudes).configure_legend(
        labelFontSize=14, 
        titleFontSize=16, 
        symbolSize=150
    ).configure_axis(
        labelFontSize=14, 
        titleFontSize=16, 
        labelFontWeight=500
    ).configure_title(fontSize=24)

#chart.save("altair_polished.html")

In [4]:
chart

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000). For information on how to plot larger datasets in Altair, see the documentation

alt.VConcatChart(...)

In [108]:
star_data.query('magnitude < 4.5')

,alt,az,airmass,moonRA,moonDec,moonSep,fieldID,objType,magnitude,mjdExpStart,risen,scheduled,observable,time_step_id,Magnitude
4,2.486895,56.425088,23.046314,289.987858,-26.213339,88.5,BD+28 4,bright star,2.06,59418.165885,True,False,False,4,2.0
5,5.689245,58.723587,10.087462,290.129043,-26.216953,88.4,BD+28 4,bright star,2.06,59418.178385,True,False,False,5,2.0
6,8.968906,60.922167,6.414432,290.267351,-26.218455,88.3,BD+28 4,bright star,2.06,59418.190885,True,False,False,6,2.0
7,12.317887,63.031030,4.687455,290.403144,-26.217762,88.2,BD+28 4,bright star,2.06,59418.203385,True,False,False,7,2.0
8,15.729008,65.060143,3.688842,290.536802,-26.214806,88.1,BD+28 4,bright star,2.06,59418.215885,True,False,False,8,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,44.821965,2.969774,1.418628,292.716024,-25.853670,109.6,BD+76 928,bright star,3.21,59418.415885,True,False,True,24,3.0
25,44.974476,1.634681,1.414844,292.874151,-25.814245,109.6,BD+76 928,bright star,3.21,59418.428385,True,False,True,25,3.0
26,45.037993,0.282775,1.413277,293.036698,-25.773480,109.5,BD+76 928,bright star,3.21,59418.440885,True,False,True,26,3.0
27,45.011827,358.927951,1.413922,293.203859,-25.731518,109.4,BD+76 928,bright star,3.21,59418.453385,True,False,True,27,3.0


In [3]:
df = pd.read_csv('../data/mjd-59418-sdss-simple-expanded.csv', index_col=0)

In [19]:
moon_pos['fieldID'] = 'moon'
moon_pos.to_csv('../data/moon-positions-mjd-59418.csv', index=False)

In [8]:
data, star_data = get_data(df)
data['fieldStatus'] = ['Observing' if x else 'Available' for x in data['scheduled']]
data.loc[data['alt'] < 40, 'fieldStatus'] = 'Unavailable'
# moon_pos = get_moon_data(data)
ts = {mjd: ii for ii, mjd in enumerate(data['mjdExpStart'].sort_values().unique())}
data['time_step_id'] = [ts[mjd] for mjd in data['mjdExpStart']]
star_data['time_step_id'] = [ts[mjd] for mjd in star_data['mjdExpStart']]

In [9]:
star_data['mag'] = round(star_data['magnitude'] * 2, 0) / 2

In [10]:
old_df = pd.read_csv('../data/mjd-59418-sdss-simple.csv', index_col=0)
moon_pos = get_moon_data(old_df)

In [11]:
moon_pos['mjdExpStart'] = round(moon_pos['mjdExpStart'], 6)
moon_pos['time_step_id'] = [ts[mjd] for mjd in moon_pos['mjdExpStart']]

moon_pos.to_csv('moon-positions-mjd-59418.csv', index=False)

In [4]:
data, star_data = get_data(df)

moon_pos = pd.read_csv('moon-positions-mjd-59418.csv')

In [5]:
'''
Use altair to make an interactive plot showing the locations of fields ove the course of the night
'''
important_cols = ['mjdExpStart', 'alt', 'az', 'moonSep', 'fieldID', 'scheduled']

directions = pd.DataFrame({"lat": [-5, -5, -5, -5], "long": [0, 90, 180, 270], "text": ["N", "E", "S", "W"]})
dir_labels = alt.Chart(directions).mark_text(fontSize=16).encode(
        longitude="long",
        latitude="lat",
        text="text")

alt_df = pd.DataFrame({"lat": [2, 30, 60, 90, 60, 30, 2], "long": [0, 0, 0, 0, 180, 180, 180], "text": ["0°", "30°", "60°", "90°", "60°", "30°", "0°"]})
alt_labels = alt.Chart(alt_df).mark_text(color='white').encode(
        longitude="long",
        latitude="lat",
        text="text")

start = data['mjdExpStart'].min()
stop = data['mjdExpStart'].max()
tstep = 0.0125

field_scale = alt.Scale(domain=('Observing', 'Available', 'Unavailable'),
                      range=["yellow", "blue", "#6E7DDB"])

slider = alt.binding_range(min=start, max=stop, step=tstep)


select_time = alt.selection_single(name="select", fields=['mjdExpStart'],
                                   bind=slider, init={'mjdExpStart': start})

moon = alt.Chart(moon_pos).mark_point(filled=True, size=400, color='#A0A0A0').encode(
    latitude='moonAlt',
    longitude='moonAz',
    tooltip=['moonPhase']
).transform_filter(
    select_time
)

stars = alt.Chart(star_data).mark_point(filled=True, color='#FFFFFF').encode(
    latitude='alt',
    longitude='az',
    size=alt.Size('mag', sort='descending', scale=alt.Scale(range=(5,50)))
).add_selection(
    select_time
).transform_filter(
    select_time
)


fields = alt.Chart(data).mark_square(opacity=1, size=120).encode(
    latitude='alt',
    longitude='az',
    color=alt.Color('fieldStatus', sort='descending', scale=field_scale),
    tooltip=['moonSep', 'fieldID']
).transform_filter(
    select_time
)

sky_map = alt.layer(
    # use the sphere of the Earth as the base layer
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#1A1A1A'
    ),
#     add a graticule for geographic reference lines
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#515151', strokeWidth=1
    ),
    stars,
    fields,
    moon,
    dir_labels,
    alt_labels,
    title="Sky with SDSS fields, looking up while facing south"
).properties(
    width=800,
    height=600
)

test =sky_map.project(
    type='azimuthalEquidistant', scale=150, translate=[400, 300], clipAngle=90, rotate=[0,-90, 180]
)

select_field = alt.selection_single(on='click', fields=['fieldID'])

alts = alt.Chart(data).mark_point().add_selection(
        select_field
    ).encode(
        x='start:T',
        y='alt:Q',
        color=alt.Color('fieldStatus', sort='descending', scale=field_scale),
        opacity=alt.condition(select_field, alt.value(0.8), alt.value(0.1))
).properties(
    width=800,
    height=200
)

rule = alt.Chart(data).mark_rule().encode(
    x = 'start'
).add_selection(
    select_time
).transform_filter(
    select_time
)

In [6]:
trial = (test & (alts + rule))

In [7]:
trial

ValueError: start encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

alt.VConcatChart(...)

In [36]:
moon_pos['fieldID']

0     moon
1     moon
2     moon
3     moon
4     moon
5     moon
6     moon
7     moon
8     moon
9     moon
10    moon
11    moon
12    moon
13    moon
14    moon
15    moon
16    moon
17    moon
18    moon
19    moon
20    moon
21    moon
22    moon
23    moon
24    moon
25    moon
26    moon
27    moon
28    moon
29    moon
30    moon
31    moon
32    moon
33    moon
34    moon
35    moon
36    moon
37    moon
38    moon
39    moon
40    moon
41    moon
42    moon
43    moon
44    moon
45    moon
46    moon
47    moon
48    moon
49    moon
Name: fieldID, dtype: object

In [41]:
alt.Chart(moon_pos).mark_point(filled=True, size=400, color='#A0A0A0', opacity=1).encode(
    latitude='moonAlt',
    longitude='moonAz',
    tooltip = ['fieldID']
)

alt.Chart(...)

In [77]:
moon_base = alt.Chart().mark_point(filled=True, size=400, color='#A0A0A0', opacity=1).encode(
    latitude='moonAlt',
    longitude='moonAz'
).transform_filter(
    select_time
)

moon_selection = alt.selection_single(on='mouseover')
moon_text = alt.Chart().mark_text().encode(
    latitude='moonAlt',
    longitude='moonAz',    
    text='fieldID',
    opacity=alt.condition(moon_selection, alt.value(1), alt.value(0))
).add_selection(
    moon_selection
)

moon_text_background = alt.Chart().mark_text(stroke='white', strokeWidth=2).encode(
    latitude='moonAlt',
    longitude='moonAz',    
    text='fieldID',
    opacity=alt.condition(moon_selection, alt.value(1), alt.value(0))
)


moon = alt.layer(
    moon_base,
    moon_text,
    moon_text_background,
    data=moon_pos
)

In [78]:
moon

alt.LayerChart(...)

In [45]:
# Interaction by mouseover on time
select_time = alt.selection_single(
#     encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none',     # empty selection includes no data points
    fields=['time_step_id'],
    init={'time_step_id': 0}
)

# Interaction on fields
select_field = alt.selection_multi(on='click', fields=['fieldID'], empty='none')

# Set up color scheme for field status

# color scheme for field status
yellow = '#E3E028' # "#6E7DDB"
field_scale = alt.Scale(domain=('Observing', 'Available', 'Unavailable'),
                      range=["yellow", "blue", '#6E7DDB'])


# Plot time against altitude
base = alt.Chart().mark_point().encode(
    x='Observation Start Time:T',
    y='alt:Q',
    color=alt.Color('fieldStatus:N', sort='descending', scale=field_scale),
    opacity=alt.condition(select_field, alt.value(1), alt.value(0.1))
).add_selection(
    select_time
).add_selection(
    select_field
).transform_filter(
    'datum.fieldStatus != "Observing"'
)

# Plot time against altitude for the currently field observed so it's always on top
observing_field_alts = alt.Chart().mark_point(filled=True).encode(
    x='Observation Start Time:T',
    y='alt:Q',
    color=alt.Color('fieldStatus:N', sort='descending', scale=field_scale, legend=None),
    opacity=alt.condition(select_field, alt.value(1), alt.value(0.5)),
    size=alt.condition(select_field, alt.value(200), alt.value(50))
).transform_filter(
    'datum.fieldStatus == "Observing"'
)

# Plot lineplot time against altitude for the currently field 
same_color_scale = alt.Scale(domain=[ID for ID in data['fieldID']], range=['#000000'] * len(data['fieldID'].unique()))
observing_field_lines = alt.Chart(data).mark_line().encode(
    x='Observation Start Time:T',
    y='alt:Q',
    color=alt.Color('fieldID', scale=same_color_scale),
).transform_filter(
    'datum.fieldStatus == "Observing"'
)

# layer all altitude plot elements together
alts = alt.layer(
    base,
    observing_field_alts,
    observing_field_lines,
    # add interactive line for mouseover
    alt.Chart().mark_rule(color='#777777').encode(
        x='Observation Start Time:T'
    ).transform_filter(select_time),
    # add text to display datetime
    alt.Chart().mark_text(align='left', dx=-375, dy=75, baseline='bottom', fontSize=14, fontWeight=300).encode(
        text=alt.Text('Observation Start Time:T', format="%Y-%m-%d %H:%M:%S")
    ).transform_filter(select_time),
    # add text to display field currently observing
    alt.Chart().mark_text(align='left', dx=-375, dy=90, baseline='bottom', fontSize=14, fontWeight=500).encode(
        text='fieldID'
    ).transform_calculate(
        fieldID = '"Obseving fieldID: " + datum.fieldID'
    ).transform_filter(select_time)
    .transform_filter(
    'datum.fieldStatus == "Observing"'
    ),
    
    data=data
).properties(
    width=800,
    height=200
)

# NSEW labels
directions = pd.DataFrame({"lat": [-3, -3, -3, -3], "long": [0, 90, 180, 270], "text": ["N", "E", "S", "W"]})
dir_labels = alt.Chart(directions).mark_text(fontSize=16).encode(
        longitude="long",
        latitude="lat",
        text="text")

# Altitude labels
alt_df = pd.DataFrame({"lat": [2, 30, 60, 90, 60, 30, 2], "long": [0, 0, 0, 0, 180, 180, 180], "text": ["0°", "30°", "60°", "90°", "60°", "30°", "0°"]})
alt_labels = alt.Chart(alt_df).mark_text(color='white').encode(
        longitude="long",
        latitude="lat",
        text="text")

moon_base = alt.Chart(moon_pos).mark_point(filled=True, size=400, color='#A0A0A0', opacity=1).encode(
    latitude='moonAlt',
    longitude='moonAz'
).transform_filter(
    select_time
)

moon_selection = alt.selection_single(on='mouseover')
moon_text = alt.Chart(moon_pos).mark_text(stroke='white', dy=12).encode(
    latitude='moonAlt',
    longitude='moonAz',    
    text='fieldID',
    opacity=alt.condition(moon_selection, alt.value(1), alt.value(0))
).transform_filter(
    select_time
).add_selection(
    moon_selection
)

moon = alt.layer(
    moon_base,
    moon_text,
    data=moon_pos
)

stars = alt.Chart(star_data).mark_point(filled=True, color='#FFFFFF').encode(
    latitude='alt',
    longitude='az',
    size=alt.Size('magnitude', sort='descending', scale=alt.Scale(range=(5,50)))
).add_selection(
    select_time
).transform_filter(
    select_time
)



fields = alt.Chart(data).mark_square(opacity=0.75, size=80).encode(
    latitude='alt',
    longitude='az',
    color=alt.Color('fieldStatus', sort='descending', scale=field_scale),
    tooltip=['moonSep', 'fieldID']
).transform_filter(
    select_time
).add_selection(
    select_field
# ).transform_filter(
#     'datum.fieldStatus != "Observing"'
)

observing_field_skymap = alt.Chart(data).mark_square(opacity=1, size=80).encode(
    latitude='alt',
    longitude='az',
    color=alt.Color('fieldStatus', sort='descending', scale=field_scale),
    tooltip=['moonSep', 'fieldID']
).transform_filter(
    select_time
).transform_filter(
    'datum.fieldStatus == "Observing"'
)

selected_field = alt.Chart(data).mark_square(opacity=1, size=90, filled=False, color='red').encode(
    latitude='alt',
    longitude='az'
).transform_filter(
    select_time
).transform_filter(
    select_field
)

sky_map = alt.layer(
    # use the sphere of the Earth as the base layer
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#1A1A1A'
    ),
#     add a graticule for geographic reference lines
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#515151', strokeWidth=1
    ),
    # stars,
    moon,
    # moon_text,
    fields,
    observing_field_skymap,
    selected_field,
    dir_labels,
    alt_labels,
    title="Sky with SDSS fields, looking up while facing south"
).properties(
    width=800,
    height=600
)

test =sky_map.project(
    type='azimuthalEquidistant', scale=178, translate=[400, 300], clipAngle=90, rotate=[0,-90, 180]
)

In [46]:
alts

alt.LayerChart(...)

In [42]:
trial = (test & alts).configure(background="#CACACA")

In [43]:
trial.configure_legend(labelFontSize=14, titleFontSize=16, symbolSize=150).configure_axis(labelFontSize=14, titleFontSize=16, labelFontWeight=500).configure_title(fontSize=24)

alt.VConcatChart(...)

In [27]:
trial.save("altair_linked_draft2.json")

In [14]:
select_field, select_time, field_scale = get_interactive_elements()

test1 = make_sky_map(data, star_data,moon_pos, select_field, select_time, field_scale)

test2 = make_alts_plot(data, select_field, select_time, field_scale)

test1 & test2